In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
from math import sin
from ROOT import gRandom
from ROOT import TFile
from ROOT import TPad, TLegend
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.29/01


In [2]:
pt_low, pt_high = 1, 12
phi_v_low, phi_v_high = 1, 16
itype = 1

In [3]:
isigma = 2
N_centr = 6
N_options = 1
N_vtx_bins = 4
centralities = ["0-88%","0-20%","20-40%","40-60%","60-80%","80-93%"]
centrality = [0,9,0,1,2,3,4,5,6,7,8,9]
colors=[1,2,4,6,root.kGreen+2]
file_path="input/"
file_name="m_ee_Run14AuAU_fifth_test_19081_650runs.root"
#"m_ee_Run14AuAU_fourth_test_19072_800runs.root"#"m_ee_Run14AuAU_first_test_19060_100runs.root"

In [4]:
hists_ep_FG_read, hists_ep_BG_read, hists_ee_FG_read, hists_ee_BG_read = [], [], [], []
hist_stat_read=[]
infile = root.TFile.Open(file_path+file_name, "read")

for iOpt in range(1,N_options+1):
    histo_name = [f"inv_mass_ee_same_arm_V{itype}_",f"inv_mass_ee_diff_arm_V{itype}_"]
    histo_name_like = [f"delt_phi_ee_same_arm_V{itype}_",f"delt_phi_ee_diff_arm_V{itype}_"]
    print(histo_name)
    hists_ep_FG_read_centr, hists_ep_BG_read_centr, hists_ee_FG_read_centr, hists_ee_BG_read_centr = [], [], [], []
    hist_stat_centr = []
    for icent in range(N_centr):
        hist_FG = infile.Get("c00_z00_r00/"+histo_name[0]+"FG12")
        hist_FG.Add(hist_FG,-1)
        hist_FG.SetDirectory(root.nullptr)
        hist_BG = infile.Get("c00_z00_r00/"+histo_name[0]+"BG12")
        hist_BG.Add(hist_BG,-1)
        hist_BG.SetDirectory(root.nullptr)
        hist_FG_like = infile.Get("c00_z00_r00/"+histo_name_like[0]+"FG11")
        hist_FG_like.Add(hist_FG_like,-1)
        hist_FG_like.SetDirectory(root.nullptr)
        hist_BG_like = infile.Get("c00_z00_r00/"+histo_name_like[0]+"BG11")
        hist_BG_like.Add(hist_BG_like,-1)
        hist_BG_like.SetDirectory(root.nullptr)
        hist_stat = infile.Get("c00_z00_r00/PoolStatistics")
        hist_stat.Add(hist_stat,-1)
        hist_stat.SetDirectory(root.nullptr)
        for i in range(centrality[icent*2],centrality[icent*2+1]+1):
            for j in range(N_vtx_bins):
                for k in range(0,2):
                    dirname=f"c0{i}_z0{j}_r00/"
                    if i>9: dirname=f"c{i}_z0{j}_r00/"
                    hist_FG_local = infile.Get(dirname+histo_name[k]+"FG12")
                    hist_BG_local = infile.Get(dirname+histo_name[k]+"BG12")
                    hist_FG_like_local = infile.Get(dirname+histo_name_like[k]+"FG11")
                    hist_BG_like_local = infile.Get(dirname+histo_name_like[k]+"BG11")
                    hist_FG_like_local.Add(infile.Get(dirname+histo_name_like[k]+"FG22"))
                    hist_BG_like_local.Add(infile.Get(dirname+histo_name_like[k]+"BG22"))
                    eta_FG_porj = hist_FG_like_local.ProjectionX(f"FG_loc_{icent}_{i*20+j}",pt_low,pt_high,phi_v_low,phi_v_high)
                    eta_BG_porj = hist_BG_like_local.ProjectionX(f"BG_loc_{icent}_{i*20+j}",pt_low,pt_high,phi_v_low,phi_v_high)

                    left = eta_FG_porj.FindBin(1.1)
                    right = eta_FG_porj.FindBin(2.3)+1

                    scale = eta_FG_porj.Integral(left,right)/(eta_BG_porj.Integral(left,right)+0.1)
                    
                    hist_FG.Add(hist_FG_local)
                    hist_BG.Add(hist_BG_local,scale)
                    hist_FG_like.Add(hist_FG_like_local)
                    hist_BG_like.Add(hist_BG_like_local,scale)
                    hist_stat.Add(infile.Get(dirname+"PoolStatistics"))

        hists_ep_FG_read_centr.append(hist_FG)
        hists_ep_BG_read_centr.append(hist_BG)
        hists_ee_FG_read_centr.append(hist_FG_like)
        hists_ee_BG_read_centr.append(hist_BG_like)
        hist_stat_centr.append(hist_stat)
    hists_ep_FG_read.append(hists_ep_FG_read_centr)
    hists_ep_BG_read.append(hists_ep_BG_read_centr)
    hists_ee_FG_read.append(hists_ee_FG_read_centr)
    hists_ee_BG_read.append(hists_ee_BG_read_centr)
    hist_stat_read.append(hist_stat_centr)

infile.Close()

['inv_mass_ee_same_arm_V1_', 'inv_mass_ee_diff_arm_V1_']


In [5]:
iOption = 0
rebin = 1
new_pt_array = array( 'd', [0.,0.05,0.1,0.15,0.25,0.35,0.45,0.55,0.65,0.725,0.8,0.875,0.95,1.0,1.05,1.15,1.3,1.5,1.75,2.0,2.25,2.5,2.75,2.9,3.0,3.05,3.1,3.15,3.2,3.3,3.45,3.55,3.65,3.75,3.85,4.5])
N_new_bins = len(new_pt_array)-1
print(N_new_bins)

35


In [6]:

c3 = root.TCanvas(f"c2",f"c2",1600,900)
c3.Divide(3,2)
legends =[]
for icentr in range(N_centr):
    c3.cd(icentr+1)
    proj = hists_ee_FG_read[iOption][icentr].ProjectionX(f"px{icentr}{iOption}",pt_low,pt_high,phi_v_low,phi_v_high)
    proj_BG = hists_ee_BG_read[iOption][icentr].ProjectionX(f"px_BG_{icentr}{iOption}",pt_low,pt_high,phi_v_low,phi_v_high)
    proj_BG.Scale(1)
    Format_Hist_total(proj,"#Delta#phi","dN/d#phi",0.22,0.15,0.03,0.1)
    proj.GetYaxis().SetTitleOffset(1.5)
    proj.SetMaximum(proj.GetMaximum()*10)
    root.gPad.SetLogy(1)
    proj.Draw("")
    proj.SetMinimum(0)
    proj.SetLineWidth(3)
    proj_BG.SetLineWidth(3)
    proj_BG.SetLineColor(2)
    proj_BG.Draw("same")
    proj.GetXaxis().SetNdivisions(505)
    proj.GetYaxis().SetNdivisions(50510)
    legends.append(root.TLegend(0.6,0.7,0.9,0.95,centralities[icentr]))
    Format_Legend(legends[-1],0.08)
    legends[-1].Draw()
        
    

c3.Draw()
c3.Print("output/m_ee/scale_BG.png")

In [7]:
c5 = root.TCanvas("ccc33","ccc33",1600,900)
iOpt = 0
pt_bins=[1,1,2,2,3,3,4,4,5,6,7,12]
c5.Divide(3,2)
legends =[]
projs,projs_BG=[],[]
for icentr in range(N_centr):
    c5.cd(icentr+1)
    kek = root.TH1D(f"my{icentr}_{iOption}",f"my{icentr}_{iOption}",N_new_bins,new_pt_array)
    projs.append(root.TH1D(f"my{icentr}_{iOption}",f"my{icentr}_{iOption}",N_new_bins,new_pt_array))
    proj = projs[-1]
    projs_BG.append(root.TH1D(f"my{icentr}_{iOption}_BG",f"my{icentr}_{iOption}_BG",N_new_bins,new_pt_array))
    proj_BG = projs_BG[-1]
    proj1 = hists_ep_FG_read[iOption][icentr].ProjectionX(f"px{icentr}{iOption}",pt_low,pt_high,phi_v_low,phi_v_high)
    proj1_BG = hists_ep_BG_read[iOption][icentr].ProjectionX(f"px_BG_{icentr}{iOption}",pt_low,pt_high,phi_v_low,phi_v_high)
    proj1_BG.Add(proj1_BG,-1)

    for ipt in range(int(len(pt_bins)/2)):
        proj2 = hists_ee_FG_read[iOption][icentr].ProjectionX(f"pxl{icentr}{ipt}",pt_bins[2*ipt],pt_bins[2*ipt+1],phi_v_low,phi_v_high)
        proj2_BG = hists_ee_BG_read[iOption][icentr].ProjectionX(f"pxl_BG_{icentr}{ipt}",pt_bins[2*ipt],pt_bins[2*ipt+1],phi_v_low,phi_v_high)
        left = proj2.FindBin(1.1)
        right = proj2.FindBin(2.1)+1
        scale = proj2.Integral(left,right)/(proj2_BG.Integral(left,right)+1)
        if abs(scale - 1) > 0.1: print(icentr, ipt, scale)
        proj1_BG.Add(hists_ep_BG_read[iOption][icentr].ProjectionX(f"pxll_BG_{icentr}{ipt}",pt_bins[2*ipt],pt_bins[2*ipt+1],phi_v_low,phi_v_high),scale)
    

    for i in range(1,1+proj.GetNbinsX()):
        for j in range(proj1.FindBin(proj.GetBinLowEdge(i)+0.001),proj1.FindBin(proj.GetBinLowEdge(i)+proj.GetBinWidth(i)+0.001)):
            proj.SetBinContent(i,proj.GetBinContent(i)+proj1.GetBinContent(j))
            proj_BG.SetBinContent(i,proj_BG.GetBinContent(i)+proj1_BG.GetBinContent(j))
        proj.SetBinContent(i,proj.GetBinContent(i)/proj.GetBinWidth(i))
        proj_BG.SetBinContent(i,proj_BG.GetBinContent(i)/proj_BG.GetBinWidth(i))
    
    proj_BG.Scale(1)
    Format_Hist_total(proj,"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.13,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,0.9)
    Format_Graph(proj_BG,49,1.5,2,3,2)
    root.gPad.SetLogy(1)
    proj.Draw("p")
    proj_BG.Draw("same p")
    #proj.Draw("same p")
    proj.GetXaxis().SetNdivisions(505)
    proj.GetYaxis().SetNdivisions(50510)
    legends.append(root.TLegend(0.6,0.7,0.9,0.95,centralities[icentr]))
    legends[-1].AddEntry(proj,"FG","pl")
    legends[-1].AddEntry(proj_BG,"BG","pl")
    Format_Legend(legends[-1],0.08)
    legends[-1].Draw()

c5.Draw()
c5.Print("output/m_ee/m_ee_newpt_FG_BG.png")

3 0 1.2126350797791219
4 0 0.0
4 5 1.1268046972812171
5 0 0.0
5 1 0.6722228902548119
5 2 0.6192717426328006
5 3 1.610932238000052
5 4 0.5465999295656412
5 5 0.0


In [8]:
c3 = root.TCanvas(f"c2",f"c2",1600,900)
sub_koeff=[1,1,1,1,0.,0.]
pt_bins=[1,1,2,2,3,3,4,4,5,6,7,12]
c3.Divide(3,2)
legends =[]
projs,projs_BG=[],[]
for icentr in range(N_centr):
    c3.cd(icentr+1)
    projs.append(root.TH1D(f"1my{icentr}_{iOption}",f"1my{icentr}_{iOption}",N_new_bins,new_pt_array))
    proj = projs[-1]
    projs_BG.append(root.TH1D(f"1my{icentr}_{iOption}_BG",f"1my{icentr}_{iOption}_BG",N_new_bins,new_pt_array))
    proj_BG = projs_BG[-1]
    proj1 = hists_ep_FG_read[iOption][icentr].ProjectionX(f"1px{icentr}{iOption}",pt_low,pt_high,phi_v_low,phi_v_high)
    proj1_BG = hists_ep_BG_read[iOption][icentr].ProjectionX(f"1px_BG_{icentr}{iOption}",pt_low,pt_high,phi_v_low,phi_v_high)
    proj1_BG.Add(proj1_BG,-1)

    for ipt in range(int(len(pt_bins)/2)):
        proj2 = hists_ee_FG_read[iOption][icentr].ProjectionX(f"1pxl{icentr}{ipt}",pt_bins[2*ipt],pt_bins[2*ipt+1],phi_v_low,phi_v_high)
        proj2_BG = hists_ee_BG_read[iOption][icentr].ProjectionX(f"1pxl_BG_{icentr}{ipt}",pt_bins[2*ipt],pt_bins[2*ipt+1],phi_v_low,phi_v_high)
        left = proj2.FindBin(1.1)
        right = proj2.FindBin(2.1)+1
        scale = proj2.Integral(left,right)/(proj2_BG.Integral(left,right)+1)
        proj1_BG.Add(hists_ep_BG_read[iOption][icentr].ProjectionX(f"1pxll_BG_{icentr}{ipt}",pt_bins[2*ipt],pt_bins[2*ipt+1],phi_v_low,phi_v_high),scale)

    for i in range(1,1+proj.GetNbinsX()):
        for j in range(proj1.FindBin(proj.GetBinLowEdge(i)+0.001),proj1.FindBin(proj.GetBinLowEdge(i)+proj.GetBinWidth(i)+0.001)):
            if abs(proj1.FindBin(proj.GetBinLowEdge(i)+proj.GetBinWidth(i)+0.001) - proj1.FindBin(proj.GetBinLowEdge(i)+0.001) - proj.GetBinWidth(i)/0.025)>0.01: 
                print(icentr,proj1.FindBin(proj.GetBinLowEdge(i)),proj1.FindBin(proj.GetBinLowEdge(i)+proj.GetBinWidth(i)),proj.GetBinLowEdge(i),proj.GetBinLowEdge(i)+proj.GetBinWidth(i))
            proj.SetBinContent(i,proj.GetBinContent(i)+proj1.GetBinContent(j))
            proj_BG.SetBinContent(i,proj_BG.GetBinContent(i)+proj1_BG.GetBinContent(j))
        proj.SetBinContent(i,proj.GetBinContent(i)/proj.GetBinWidth(i))
        proj_BG.SetBinContent(i,proj_BG.GetBinContent(i)/proj_BG.GetBinWidth(i))
        
    proj_BG.Scale(1)
    for ibin in range(1,1+proj.GetNbinsX()):
        proj.SetBinError(ibin,(proj.GetBinContent(ibin))**0.5)
        proj.SetBinContent(ibin,proj.GetBinContent(ibin)-sub_koeff[icentr]*proj_BG.GetBinContent(ibin))
    Format_Hist_total(proj,"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.13,0.13,0.01,0.01,0.07,0.06,21,1,4,3,4,0.85,0.9)
    root.gPad.SetLogy(1)
    proj.Draw("")
    proj.SetLineWidth(3)
    proj.GetXaxis().SetNdivisions(505)
    proj.GetYaxis().SetNdivisions(50510)
    legends.append(root.TLegend(0.6,0.7,0.9,0.95,centralities[icentr]))
    Format_Legend(legends[-1],0.08)
    legends[-1].AddEntry(proj,"FG-BG","pl")
    legends[-1].Draw()

    print(round(hist_stat_read[iOption][icentr].GetBinContent(2)/2e9,3),"*10^9")
    

c3.Draw()
c3.Print("output/m_ee/m_ee_newpt_FG.png")

6.546 *10^9
1.473 *10^9
1.447 *10^9
1.427 *10^9
1.399 *10^9
0.799 *10^9


In [9]:
#!jupyter nbconvert --to html m_ee_plotter.ipynb --HTMLExporter.theme=dark;